In [1]:
!pip install transformers[sentencepiece] datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4",lang1="ar", lang2="en")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 116239
    })
})

In [3]:
raw_datasets["train"][0]


{'id': '0', 'translation': {'ar': 'مانع الإعلانات', 'en': 'Adblock'}}

In [4]:

split_datasets= raw_datasets["train"].train_test_split(train_size=0.9,seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 104615
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 11624
    })
})

In [5]:
split_datasets['validation'] = split_datasets.pop('test')
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 104615
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 11624
    })
})

In [6]:
split_datasets['validation'][5]

{'id': '21984',
 'translation': {'ar': 'انقر "اختبر" لكي تسمع جرس النظام بإعداداتك المغيّرة.',
  'en': 'Click "Test" to hear how the system bell will sound using your changed settings.'}}

In [7]:
from transformers import pipeline

model_checkpoint = 'Helsinki-NLP/opus-mt-ar-en'
translator = pipeline("translation", model=model_checkpoint)
translator("أهلاً بكم في هذا الدرس")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Welcome to this lesson.'}]

In [8]:
split_datasets['train'][0]

{'id': '3214', 'translation': {'ar': 'العرض:', 'en': 'Width'}}

In [9]:
from datasets import load_dataset

def extract_langauges(example):
  inputs = [ex['ar'] for ex in example['translation'] ]
  target = [ex['en'] for ex in example['translation'] ]

  return {'inputs':inputs,'target':target}

new_datasets = raw_datasets.map(extract_langauges,batched=True)
new_datasets

Map:   0%|          | 0/116239 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'inputs', 'target'],
        num_rows: 116239
    })
})

In [10]:
new_datasets = new_datasets.remove_columns(['id','translation'])
new_datasets

DatasetDict({
    train: Dataset({
        features: ['inputs', 'target'],
        num_rows: 116239
    })
})

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
ar_setence = split_datasets['train'][1]['translation']['ar']
en_setence = split_datasets['train'][1]['translation']['en']

inputs = tokenizer(ar_setence,text_target=en_setence)
inputs


{'input_ids': [5477, 4264, 546, 0], 'attention_mask': [1, 1, 1, 1], 'labels': [36225, 650, 20807, 0]}

In [13]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

['▁صورة', '▁فلا', 'ش', '</s>']

In [14]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["ar"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [15]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 104615
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 11624
    })
})

In [16]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/104615 [00:00<?, ? examples/s]

Map:   0%|          | 0/11624 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 104615
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11624
    })
})

In [18]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained( model_checkpoint)

In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1,3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [21]:
batch["decoder_input_ids"]

tensor([[62833, 36225,   650, 20807],
        [62833, 30402,   182,     0]])

In [22]:
[tokenizer.decode(i)for i in batch["input_ids"]]

['صورة فلاش</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
 'تراجع لصق@ item: inmenu undo sort</s>']

In [23]:
[tokenizer.decode(i)for i in batch["labels"]]

['Photo Flash</s>', 'Paste</s> <unk>']

In [24]:
[tokenizer.decode(i)for i in batch["decoder_input_ids"]]

['<pad> Photo Flash', '<pad> Paste</s>']

In [25]:
!pip install sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [26]:
import evaluate

metric = evaluate.load("sacrebleu")

In [27]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [28]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [40]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    if preds is None or len(preds) == 0:
        print("Predictions are empty")
        return {"bleu": 0.0}  # Default value for empty predictions

    try:
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    except Exception as e:
        print(f"Error during decoding: {e}")
        return {"bleu": 0.0}

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [30]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    f'shark-finetuned-kde4-ar-en',
    evaluation_strategy = "no",
    save_strategy='epoch',
    learning_rate =2e-5,
    per_device_train_batch_size =32,
    per_device_eval_batch_size =64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [42]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,2.630800
1000,2.434900
1500,2.333400
2000,2.236600
2500,2.249400
3000,2.243900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}


TrainOutput(global_step=3270, training_loss=2.3374108177441704, metrics={'train_runtime': 572.8423, 'train_samples_per_second': 182.624, 'train_steps_per_second': 5.708, 'total_flos': 1383488406355968.0, 'train_loss': 2.3374108177441704, 'epoch': 1.0})

In [66]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}


events.out.tfevents.1730207635.4823c3f64354.730.0:   0%|          | 0.00/7.44k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmedshark/shark-finetuned-kde4-ar-en/commit/d3db624422e96399055597123b1328a557b742f6', commit_message='Training complete', commit_description='', oid='d3db624422e96399055597123b1328a557b742f6', pr_url=None, pr_revision=None, pr_num=None)

In [44]:
from torch.utils.data import DataLoader
tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    collate_fn=data_collator,
    batch_size=8
)

In [45]:
modelfromscratch = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [77]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [78]:
from accelerate import Accelerator

accelerator = Accelerator()

model , optimizer , train_dataloader , eval_dataloader = accelerator.prepare(modelfromscratch,optimizer,train_dataloader,eval_dataloader)

In [52]:
from transformers import get_scheduler


num_train_epochs =3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [67]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "shark-finetuned-kde4-ar-en-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'ahmedshark/shark-finetuned-kde4-ar-en-accelerate'

In [64]:
notebook_login()

In [69]:
output_dir = "shark-finetuned-kde4-ar-en-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/ahmedshark/shark-finetuned-kde4-ar-en-accelerate into local empty directory.


In [73]:
def postprocess(prediction,labels):
  predicitons = prediction.cpu().numpy()
  labels = labels.cpu().numpy()

  decoded_preds = tokenizer.batch_decode(predicitons,skip_special_tokens=True)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]
  return decoded_preds, decoded_labels


In [72]:
tokenizer.pad_token_id

62833

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [81]:
model_dir = "ahmedshark/shark-finetuned-kde4-ar-en"
input_text = 'مرحبا انا احمد الشرقاوي وهذا نموذج الترجمه الخاص بي'
model = pipeline('translation',model_dir)
model( input_text )

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': "Hi, I'm Ahmed Al-Sharqawi, and this is my translation form."}]

In [82]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
translation

"Hi, I'm Ahmed Al-Sharqawi, and this is my translation form."